In [37]:
import pandas as pd 
import numpy as np
import warnings

data = pd.read_csv(r"F:\M.Tech_CollgeMaterials\IronCode\SeriesRecommdnSystem\dataset\netflix_titles.csv")

In [38]:
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [39]:
data.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [40]:
data = data.drop('director',axis=1)
data['rating'].fillna(data['rating'].mode()[0], inplace=True)
data['duration'].fillna(data['duration'].mode()[0], inplace = True )
data = data.drop('date_added',axis=1)


C:\Users\sayed\AppData\Local\Temp\ipykernel_17936\740347890.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['rating'].fillna(data['rating'].mode()[0], inplace=True)
C:\Users\sayed\AppData\Local\Temp\ipykernel_17936\740347890.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a cop

In [41]:
data['country'].fillna('Unknown', inplace=True)
data['cast'].fillna('Unknown',inplace=True)


C:\Users\sayed\AppData\Local\Temp\ipykernel_17936\1216682378.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['country'].fillna('Unknown', inplace=True)
C:\Users\sayed\AppData\Local\Temp\ipykernel_17936\1216682378.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [42]:
data.isnull().sum()

show_id         0
type            0
title           0
cast            0
country         0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [43]:
data['text'] = (
    data['title'] + ' ' +
    data['description'] + ' ' +
    data['listed_in'] + ' ' + 
    data['cast'] + ' ' +
    data['country']
)


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(data['text'])


In [45]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [46]:
series_indices = pd.Series(data.index, index=data['title']).drop_duplicates()

def recommend_series(title, n=5):
    idx = series_indices.get(title)
    if idx is None:
        return "Title not found."
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    recommended_indices = [i[0] for i in sim_scores]
    return data['title'].iloc[recommended_indices]


In [47]:
print(recommend_series("Stranger Things", n=5))


5200    Beyond Stranger Things
6703               Equilibrium
1335                The Sinner
1240                Safe Haven
1487           Sakho & Mangane
Name: title, dtype: object
